#Aggregating our Silver Layer Data into Gold Fact Tables

Creating our gold layer table if it doesn't exist (for first run)

In [0]:
%sql
USE tj_db;
CREATE TABLE IF NOT EXISTS tj_db.gold_fact_revenue_by_quarter
(
  Year STRING
, Quarter STRING
, Total_Revenue DOUBLE
, COGS DOUBLE
, Profit DOUBLE
)

USING delta

Merging data into our table and performing aggregations 

In [0]:
%sql
WITH RecalcAggregation AS (
  SELECT 
    Year, 
    Quarter, 
    BROUND(SUM(Total_Revenue),2) AS Total_Revenue, 
    BROUND(SUM(COGS),2) AS COGS, 
    BROUND(SUM(Profit),2) AS Profit
  FROM tj_db.silver_tj_fact_revenue_profit_recalc
  GROUP BY Year, Quarter
)
MERGE INTO tj_db.gold_fact_revenue_by_quarter AS gold
USING RecalcAggregation AS recalc
ON recalc.Year = gold.Year AND recalc.Quarter = gold.Quarter 
WHEN MATCHED THEN UPDATE SET 
    gold.Total_Revenue = recalc.Total_Revenue, 
    gold.COGS = recalc.COGS,
    gold.Profit = recalc.Profit
WHEN NOT MATCHED THEN INSERT (Year, Quarter, Total_Revenue, COGS, Profit)
  VALUES (recalc.Year, 
  recalc.Quarter, 
  recalc.Total_Revenue, 
  recalc.COGS, 
  recalc.Profit)

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
1,0,0,1
